Data loading :

In [ ]:
import pandas as pd

# Importing the data from the sample_data directory
try:
  df_en = pd.read_csv("./sample_data/train_en.csv")
  df_it = pd.read_csv("./sample_data/train_it.csv")
  df_es = pd.read_csv("./sample_data/train_es.csv")
  print("File loaded successfully")
except FileNotFoundError:
  print("File not found in /sample_data/. Please check the file path.")
  print("Available files:")
  !ls /sample_data/


File not found in /sample_data/. Please check the file path.
Available files:
ls: cannot access '/sample_data/': No such file or directory


Getting a sense of the data :

In [ ]:
#Exploring the data and its distribution:

print(df_en['label'].value_counts())
ratio_en = df_en['label'].value_counts()[1] / df_en['label'].value_counts()[0]
print("ratio for english : ", round(ratio_en, 3))

print(df_it['label'].value_counts())
ratio_it = df_it['label'].value_counts()[1] / df_it['label'].value_counts()[0]
print("ratio for italian : ", round(ratio_it, 3))


print(df_es['label'].value_counts())
ratio_es = df_es['label'].value_counts()[1] / df_es['label'].value_counts()[0]
print("ratio for spanish : ", round(ratio_es, 3))


label
0    938
1     88
Name: count, dtype: int64
ratio for english :  0.094
label
0    879
1    207
Name: count, dtype: int64
ratio for italian :  0.235
label
0    743
1    133
Name: count, dtype: int64
ratio for spanish :  0.179


In [ ]:
# Set pandas display options to show the full content of the 'text' column
pd.set_option('display.max_colwidth', None)

# Now display the filtered DataFrame again to see the full text
print(df_en[df_en["label"] == 1])

           id  \
2     en_1312   
29    en_1108   
61    en_1187   
69     en_766   
72     en_956   
...       ...   
944   en_1229   
954    en_519   
993    en_427   
1008   en_890   
1009   en_752   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    text  \
2                                                                                                                                                                                                                                                                  And to be fair, getting triggered by slurs is just stupid to

In [ ]:
df_en['text_length'] = df_en['text'].str.len()
df_en.head(15)

,id,text,label,lang,text_length
0,en_1021,"I've never heard anyone use the word ""faggot"" and not sound like an insecure 13 year old.",0,en,90
1,en_1496,"So you don't see the slighest problem of somebody calling other people ""faggots""?",0,en,81
2,en_1312,"And to be fair, getting triggered by slurs is just stupid to begin with. Yeah I'm a faggot, yeah I'm gay. I'm more offended by being called homosexual because it has more syllables and thus takes more time to say.",1,en,213
3,en_469,"I kinda feel like it's saying ""the faggot community"" haha. It's weird. Im not necessarily losing sleep or offended but it seems like a poor choice compared to gay, LGBT or GSM (personal favorite)",0,en,195
4,en_565,"Homophobia, racism, and the resulting endless stream of antigay and racist language and banter is the main reason I do not participate in any games that are dominated by online interaction. I would never use a game's Chat window, much less an actual voice to voice system. \n\nThis is what I think of when I hear of all of the ""features"" like resource sharing, trade, and chatting in SimCity and other games: someone is going to call me fag or nigger in about 3 minutes. \n\nI don't need that shit. Reddit is bad enough! I just want to build my city by myself, and use my games on planes, trains, or visiting family in the boonies where there is no Internet. But nope, gotta be online all the time because your ""vision"" for a God Game is all about social networking. No thanks.",0,en,777
5,en_1610,@USER ew queer white thirsty bitch,0,en,34
6,en_1189,Or... \nHim: Look at those two faggots. \nYou: I know. Aren't they amazing? I wonder if I can get their numbers.,0,en,112
7,en_181,"&gt; If 50 of them being slaughtered by an Islamic maniac didn't do it, I think we have to give up on the gay community and realize they care more about rainbows, butterflies and degenerate Pride parades than actually staying alive. Obviously I would never suggest that their gay brains are, like women, incapable of reason. (Just kidding ladies!) But what else is it going to take? Sad but true: I may be the only based faggot ever!\n\nI'm starting to think he may be just acting. Nobody can seriously believe this shit.\n\n",0,en,521
8,en_585,Fight = alpha\n\nFlight = beta\n\nFreeze = faggot,0,en,45
9,en_845,"Too bad you're already going down in history as a fag nigger. I've said it before, and I'll say it again, but when Bernie gets elected he won't have any tolerance for fag niggers like you. \n",0,en,191


Data analysis and preprocessing:

In [ ]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the datasets
# Ensure these files are uploaded to your Colab Files section
files = ['./sample_data/train_en.csv',
         './sample_data/train_es.csv',
         './sample_data/train_it.csv']
datasets = {}

for f in files:
    try:
        # Fix: Extract filename first to avoid splitting path directories
        filename = os.path.basename(f) # e.g., 'train_en.csv'
        lang = filename.split('_')[1].split('.')[0] # extract 'en', 'es', 'it'
        datasets[lang] = pd.read_csv(f)
        print(f"Successfully loaded {f} (Language: {lang})")
    except FileNotFoundError:
        print(f"⚠️ Error: {f} not found. Please upload it to Colab.")
    except Exception as e:
        print(f"⚠️ Error processing {f}: {e}")

# 2. Define Cleaning Function
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove HTML entities like &gt; (common in tweet data)
    text = re.sub(r'&[a-z]+;', '', text)
    # Remove URLs (they rarely carry sentiment for this specific task)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    # Remove extra whitespace/newlines
    text = re.sub(r'\s+', ' ', text).strip()
    # Note: We KEEP @USER and hashtags (#) as they provide valuable context
    return text

# 3. Run Analysis & Preprocessing
for lang, df in datasets.items():
    print(f"\n{'='*10} {lang.upper()} DATASET {'='*10}")

    # --- Basic Stats ---
    print(f"Total Rows: {df.shape[0]}")
    print(f"Label Distribution:\n{df['label'].value_counts()}")

    # --- Task B Check: Missing Bios ---
    if 'bio' in df.columns:
        missing_bio = df['bio'].isnull().sum()
        print(f"Missing Bios: {missing_bio} ({missing_bio/len(df)*100:.1f}%)")
        # Fill missing bios with empty string for Task B
        df['bio_cleaned'] = df['bio'].fillna("").apply(clean_text)

    # --- Text Length Analysis ---
    # Helping us decide on tokenizer max_length
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))
    print(f"Max Word Count: {df['word_count'].max()}")
    print(f"Avg Word Count: {df['word_count'].mean():.1f}")

    # --- Apply Cleaning ---
    df['cleaned_text'] = df['text'].apply(clean_text)

    # --- Save Processed Data ---
    output_filename = f'./processed_data_{lang}.csv'
    df.to_csv(output_filename, index=False)
    print(f"✅ Saved processed data to {output_filename}")

    # Display a sample
    print(f"\nSample Cleaned Entry:")
    print(df[['cleaned_text', 'label']].iloc[0].values)

Successfully loaded /content/drive/MyDrive/Academical Research Project/multipride_data/train_en.csv (Language: en)
Successfully loaded /content/drive/MyDrive/Academical Research Project/multipride_data/train_es.csv (Language: es)
Successfully loaded /content/drive/MyDrive/Academical Research Project/multipride_data/train_it.csv (Language: it)

========== EN DATASET ==========
Total Rows: 1026
Label Distribution:
label
0    938
1     88
Name: count, dtype: int64
Max Word Count: 148
Avg Word Count: 38.1
✅ Saved processed data to /content/drive/MyDrive/Academical Research Project/processed_data_en.csv

Sample Cleaned Entry:
['I\'ve never heard anyone use the word "faggot" and not sound like an insecure 13 year old.'
 np.int64(0)]

========== ES DATASET ==========
Total Rows: 876
Label Distribution:
label
0    743
1    133
Name: count, dtype: int64
Missing Bios: 49 (5.6%)
Max Word Count: 61
Avg Word Count: 33.5
✅ Saved processed data to /content/drive/MyDrive/Academical Research Project/pr

In [ ]:
!pip install transformers datasets scikit-learn accelerate -q

First training of the model:

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from torch import nn

# ================= CONFIGURATION =================
# DISABLE WANDB LOGGING (The fix for your error)
os.environ["WANDB_DISABLED"] = "true"

LANGUAGE = 'it'  # Change to 'es' or 'en' to train other models
MODEL_NAME = "xlm-roberta-base"
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-5
# =================================================

# 1. Load and Split Data
filename = f'./processed_data_{LANGUAGE}.csv'
try:
    df = pd.read_csv(filename)
except FileNotFoundError:
    df = pd.read_csv('processed_data.csv')
    print("⚠️ Loaded 'processed_data.csv' as fallback.")

df = df.dropna(subset=['cleaned_text', 'label'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['cleaned_text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

print(f"Training on {len(train_texts)} samples, Validating on {len(val_texts)} samples.")

# 2. Calculate Class Weights
class_counts = np.bincount(train_labels)
total_samples = len(train_labels)
class_weights = torch.tensor([total_samples / (2 * c) for c in class_counts], dtype=torch.float)
print(f"Class Weights (0, 1): {class_weights}")

# 3. Tokenization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN)

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

# 4. Custom Dataset Class
class MultiPrideDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MultiPrideDataset(train_encodings, train_labels)
val_dataset = MultiPrideDataset(val_encodings, val_labels)

# 5. Custom Trainer with Weighted Loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# 6. Metrics Function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_macro = f1_score(labels, preds, average='macro')
    return {'macro_f1': f1_macro}

# 7. Initialize Model & Training
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

training_args = TrainingArguments(
    output_dir=f'./results_{LANGUAGE}',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    save_total_limit=2,
    report_to="none" # Also explicitly tells it not to report to wandb
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 8. TRAIN!
print("\nStarting Training...")
trainer.train()

# 9. Evaluation
print("\nFinal Validation Results:")
eval_results = trainer.evaluate()
print(eval_results)

preds = trainer.predict(val_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)
print("\nClassification Report:")
print(classification_report(val_labels, pred_labels, target_names=['Not Reclaimed (0)', 'Reclaimed (1)']))

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

Interpretation of the results :
The model is almost perfect at recognizing non reclaimed terms as the F1 measure shows.
We can also say that the weighted class function pretty well at preventing overfitting, even though the model started to memorize at epoch number 4.

Data augmentation:

In [ ]:
# 1. Select examples to show the LLM
# We will take 5 random examples of "Reclaimed" (Label 1) tweets from the Italian dataset
import random

reclaimed_examples = df[df['label'] == 1]['cleaned_text'].sample(5, random_state=42).tolist()

print("Copy this prompt into Gemini:\n")
print("I am training a classifier to detect 'reclaimed language' in the LGBTQ+ community in Italian.")
print("Here are 5 real examples from my training data where the label is 'Reclaimed' (1):")
for i, text in enumerate(reclaimed_examples):
    print(f"{i+1}. {text}")
print("\nPlease generate 20 NEW, synthetic Italian tweets that use similar LGBTQ+ slurs or terms (like frocio, ricchione, lella, queer) in a RECLAIMED, positive, or self-identifying way.")
print("The tone should vary (ironic, proud, angry, community-focused).")
print("Format the output as a simple list.")

Copy this prompt into Gemini:

I am training a classifier to detect 'reclaimed language' in the LGBTQ+ community in Italian.
Here are 5 real examples from my training data where the label is 'Reclaimed' (1):
1. no vabbè appena iniziato american horror stories e c’è già la coppia forcia e la coppia saffica per ora NESSUNA coppia eterosessuale in vista we love to see it
2. non questo ricchione della mia uni che ha fatto questa canzone chiamata triangoli rosa (e già qui vabbè) ed oggi ha usato il pretesto della giornata della memoria per sponsorizzarla #noddlzan
3. e comunque i galio hanno detto diritti froci
4. il la spezia pride non so se verrei pestato prima per essere frocio o per essere carrarino .
5. Sto guardando la saga degli X-Men su #disneyplus e è una figata la metafora dei mutanti=froci, queer, neri ecc. Certo, almeno scene super poracce

Please generate 20 NEW, synthetic Italian tweets that use similar LGBTQ+ slurs or terms (like frocio, ricchione, lella, queer) in a RECLAIME

Data augmentation with pregenerated list

In [ ]:
import pandas as pd

# These are synthetic examples of "Reclaimed" (Label 1) tweets
# designed to boost your model's recall.
synthetic_data = [
    ("Finalmente ho comprato la mia bandiera trans, sono così felice di portarla al pride! 🏳️‍⚧️", 1),
    ("Essere frocio non è una vergogna, è il mio superpotere. #orgoglio", 1),
    ("Stasera serata queer con le mie amiche lelle, non vedo l'ora!", 1),
    ("Mio cugino ha fatto coming out come trans, sono fierissimo di lui.", 1),
    ("Non mi insulti se mi chiami ricchione, descrivi solo la mia favolosità.", 1),
    ("Sì, sono una camionista lella e allora? Problemi?", 1),
    ("La cultura queer è arte, mettetevelo in testa.", 1),
    ("Oggi mi sento una frocia pazzesca, spostatevi tutti.", 1),
    ("Ho pianto di gioia al pride, vedere così tante persone trans felici è stato bellissimo.", 1),
    ("Mia nonna che mi chiede se ho trovato il fidanzatino... se sapesse che sono la regina dei froci!", 1),
    ("Rivendico la parola 'frocio'. Me la riprendo e la uso con amore.", 1),
    ("Basta nascondersi, siamo queer, siamo qui e non ce ne andiamo.", 1),
    ("Supportate i vostri amici trans, sempre.", 1),
    ("Che bello essere una lella libera in un mondo di giudiconi.", 1),
    ("Orgogliosamente terrone e orgogliosamente frocio.", 1),
    ("Non c'è niente di male nell'essere diversi, la normalità è sopravvalutata. #queerlife", 1),
    ("Ho messo lo smalto rosa per solidarietà con i miei fratelli femminielli.", 1),
    ("Le persone non binarie sono valide, mettetevelo in quella testa dura.", 1),
    ("Oggi mi sento proprio una checca isterica e lo adoro.", 1),
    ("Il mio ragazzo trans è la persona più coraggiosa che conosca.", 1),
    ("Lesbica e fiera, non mi avrete mai come volete voi.", 1),
    ("Dicono che è 'contro natura', io dico che essere queer è la cosa più naturale del mondo.", 1),
    ("Ho decorato tutta la casa arcobaleno, scoppiate omofobi!", 1),
    ("Essere 'quella strana' della famiglia è il mio ruolo preferito. #queer", 1),
    ("Saluti da un gruppo di froci in vacanza! 🌈", 1),
    ("Mi hanno urlato contro, ma io ho sorriso. La mia identità trans non si tocca.", 1),
    ("Siamo la rivoluzione colorata. #transrights", 1),
    ("Non sono confusa, sono bisessuale. C'è differenza.", 1),
    ("Adoro quando la mia ragazza mi chiama 'la mia piccola lella'.", 1),
    ("Porto questa spilla rainbow con onore.", 1)
]

# Create DataFrame and save
df_synthetic = pd.DataFrame(synthetic_data, columns=['cleaned_text', 'label'])
df_synthetic['lang'] = 'it' # Set language
df_synthetic.to_csv('synthetic_data_it.csv', index=False)

print(f"✅ Created 'synthetic_data_it.csv' with {len(df_synthetic)} new examples.")

✅ Created 'synthetic_data_it.csv' with 30 new examples.


In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from torch import nn

# --- CONFIG ---
os.environ["WANDB_DISABLED"] = "true"
LANGUAGE = 'it'
MODEL_NAME = "xlm-roberta-base"
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 5  # Increased slightly to accommodate new data
LEARNING_RATE = 2e-5

# 1. Load Real Data
try:
    df_real = pd.read_csv(f'./processed_data_{LANGUAGE}.csv')
except:
    df_real = pd.read_csv('processed_data.csv') # Fallback
df_real = df_real.dropna(subset=['cleaned_text', 'label'])

# 2. Split Real Data (The "Golden Rule" of Validation)
# We split BEFORE adding synthetic data so our validation score is honest.
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_real['cleaned_text'].tolist(),
    df_real['label'].tolist(),
    test_size=0.2,
    stratify=df_real['label'],
    random_state=42
)

# 3. Load and Inject Synthetic Data (Train Set ONLY)
try:
    df_syn = pd.read_csv(f'synthetic_data_{LANGUAGE}.csv')
    print(f"💉 Injecting {len(df_syn)} synthetic examples into Training Set...")

    # Add to training lists
    train_texts.extend(df_syn['cleaned_text'].tolist())
    train_labels.extend(df_syn['label'].tolist())
except FileNotFoundError:
    print("⚠️ No synthetic data found. Training on real data only.")

print(f"Training Size: {len(train_texts)} | Validation Size: {len(val_texts)}")

# 4. Tokenization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=MAX_LEN)
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=MAX_LEN)

# 5. Class Weights (Recalculate with new data)
class_counts = np.bincount(train_labels)
total_samples = len(train_labels)
class_weights = torch.tensor([total_samples / (2 * c) for c in class_counts], dtype=torch.float)
print(f"New Class Weights: {class_weights}")

# 6. Dataset Class
class MultiPrideDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = MultiPrideDataset(train_encodings, train_labels)
val_dataset = MultiPrideDataset(val_encodings, val_labels)

# 7. Trainer with Weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {'macro_f1': f1_score(labels, preds, average='macro')}

# 8. Train
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

training_args = TrainingArguments(
    output_dir=f'./results_{LANGUAGE}_augmented',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    save_total_limit=2,
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# 9. Evaluate & Compare
print("\n--- AUGMENTED RESULTS ---")
preds = trainer.predict(val_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)
print(classification_report(val_labels, pred_labels, target_names=['Not Reclaimed (0)', 'Reclaimed (1)']))

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

Training all models sequentially

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn

# --- EMERGENCY CONFIGURATION (DistilBERT) ---
os.environ["WANDB_DISABLED"] = "true"
MODEL_NAME = "distilbert-base-multilingual-cased"  # <--- Smaller, faster model
BATCH_SIZE = 16
EPOCHS = 3          # Reduced from 4 to 3 to save time
LEARNING_RATE = 2e-5
MAX_LEN = 128

# Function to prepare data (handles both Task A and Task B)
def prepare_data(lang, task_type='A'):
    # Load raw data to ensure we have 'bio' for Task B
    df = pd.read_csv(f'./sample_data/train_{lang}.csv')

    # Text Cleaning (Minimal)
    df['text'] = df['text'].fillna("").astype(str)
    if 'bio' in df.columns:
        df['bio'] = df['bio'].fillna("").astype(str)
    else:
        df['bio'] = ""

    # Task B: Concatenate Text + Bio
    if task_type == 'B':
        print(f"🔗 Preparing Task B (Context) for {lang}...")
        df['input_text'] = df['text'] + " [SEP] " + df['bio']
    else:
        print(f"📄 Preparing Task A (Text Only) for {lang}...")
        df['input_text'] = df['text']

    # Stratified Split
    return train_test_split(
        df['input_text'].tolist(),
        df['label'].tolist(),
        test_size=0.2,
        stratify=df['label'],
        random_state=42
    )

# The Trainer Class
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {'macro_f1': f1_score(labels, preds, average='macro')}

# --- TRAINING PIPELINE ---
def train_model(lang, task_type, output_dir):
    # 1. Prepare Data
    train_texts, val_texts, train_labels, val_labels = prepare_data(lang, task_type)

    # 2. Tokenize
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_enc = tokenizer(train_texts, padding=True, truncation=True, max_length=MAX_LEN)
    val_enc = tokenizer(val_texts, padding=True, truncation=True, max_length=MAX_LEN)

    # 3. Create Datasets
    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, enc, lab): self.enc, self.lab = enc, lab
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.enc.items()}
            item['labels'] = torch.tensor(self.lab[idx])
            return item
        def __len__(self): return len(self.lab)

    train_ds = MyDataset(train_enc, train_labels)
    val_ds = MyDataset(val_enc, val_labels)

    # 4. Calculate Weights
    counts = np.bincount(train_labels)
    # Aggressive weighting for English to fix 0 recall
    if lang == 'en':
        weights = torch.tensor([1.0, 15.0], dtype=torch.float) # Force it to pay attention to class 1
    else:
        weights = torch.tensor([len(train_labels) / (2 * c) for c in counts], dtype=torch.float)

    print(f"⚖️ Class Weights: {weights}")

    # 5. Train
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        learning_rate=LEARNING_RATE,
        save_total_limit=1,
        report_to="none"
    )

    trainer = WeightedTrainer(
        class_weights=weights,
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics
    )

    trainer.train()
    trainer.save_model(output_dir) # Save the best model explicitly
    tokenizer.save_pretrained(output_dir) # Save tokenizer for inference

    print(f"✅ Model {lang.upper()}-{task_type} Saved!")
    return trainer.evaluate()

# --- EXECUTE PLAN ---
# 1. English Rescue (Task A)
print("\n🚨 STARTING ENGLISH RESCUE RUN...")
train_model('en', 'A', './model_A_en')

# 2. Spanish Baseline (Task A)
print("\n🇪🇸 STARTING SPANISH TASK A...")
train_model('es', 'A', './model_A_es')

# 3. Italian Context (Task B)
print("\n🇮🇹 STARTING ITALIAN TASK B...")
train_model('it', 'B', './model_B_it')

# 4. Spanish Context (Task B)
print("\n🇪🇸 STARTING SPANISH TASK B...")
train_model('es', 'B', './model_B_es')

# Note: We skip training model_A_it because you already have a good one.
# We will assume your previous best model is saved. If not, re-run it quickly.


🚨 STARTING ENGLISH RESCUE RUN...
📄 Preparing Task A (Text Only) for en...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

⚖️ Class Weights: tensor([ 1., 15.])


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
!rm -rf ~/.cache/huggingface

In [ ]:
import os
import shutil
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch import nn
import torch
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
os.environ["WANDB_DISABLED"] = "true"
MODEL_ID = "distilbert-base-multilingual-cased" # Smaller model (500MB vs 1.5GB)
LOCAL_MODEL_PATH = "/content/local_model_cache" # <--- SAVING TO LOCAL DISK (FAST)
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LEN = 128

# 1. FORCE DOWNLOAD TO LOCAL DISK (Bypasses Drive Freeze)
if os.path.exists(LOCAL_MODEL_PATH):
    shutil.rmtree(LOCAL_MODEL_PATH) # Clear potential corrupted cache

print(f"🚀 Downloading {MODEL_ID} to LOCAL disk ({LOCAL_MODEL_PATH})...")
# This uses the robust hub downloader which handles resuming and avoids Drive I/O
snapshot_download(repo_id=MODEL_ID, local_dir=LOCAL_MODEL_PATH)
print("✅ Download Complete! No 99% stall.")

# 2. Data Preparation Function
def prepare_data(lang, task_type='A'):
    try:
        df = pd.read_csv(f'train_{lang}.csv')
    except:
        # Fallback for urgency
        print(f"⚠️ Warning: Could not find train_{lang}.csv, checking drive...")
        df = pd.read_csv(f'./sample_data/train_{lang}.csv')

    df['text'] = df['text'].fillna("").astype(str)
    if 'bio' in df.columns:
        df['bio'] = df['bio'].fillna("").astype(str)
    else:
        df['bio'] = ""

    if task_type == 'B':
        print(f"🔗 Task B ({lang}): Text + Bio")
        df['input_text'] = df['text'] + " [SEP] " + df['bio']
    else:
        print(f"📄 Task A ({lang}): Text Only")
        df['input_text'] = df['text']

    return train_test_split(
        df['input_text'].tolist(),
        df['label'].tolist(),
        test_size=0.2,
        stratify=df['label'],
        random_state=42
    )

# 3. Training Function
def train_fast_model(lang, task_type):
    train_texts, val_texts, train_labels, val_labels = prepare_data(lang, task_type)

    # Load from LOCAL path
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
    train_enc = tokenizer(train_texts, padding=True, truncation=True, max_length=MAX_LEN)
    val_enc = tokenizer(val_texts, padding=True, truncation=True, max_length=MAX_LEN)

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, enc, lab): self.enc, self.lab = enc, lab
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.enc.items()}
            item['labels'] = torch.tensor(self.lab[idx])
            return item
        def __len__(self): return len(self.lab)

    train_ds = MyDataset(train_enc, train_labels)
    val_ds = MyDataset(val_enc, val_labels)

    # Weights for class imbalance
    counts = np.bincount(train_labels)
    # Stronger weights for English
    if lang == 'en': weights = torch.tensor([1.0, 10.0], dtype=torch.float)
    else: weights = torch.tensor([len(train_labels) / (2 * c) for c in counts], dtype=torch.float)

    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = nn.CrossEntropyLoss(weight=weights.to(model.device))
            return (loss_fct(logits.view(-1, 2), labels.view(-1)), outputs) if return_outputs else loss_fct(logits.view(-1, 2), labels.view(-1))

    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_PATH, num_labels=2)

    output_dir = f'./results_{lang}_{task_type}'
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        save_strategy="no", # Don't save checkpoints to save time/space
        eval_strategy="epoch",
        learning_rate=LEARNING_RATE,
        report_to="none"
    )

    trainer = WeightedTrainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')})

    print(f"🏃 Training {lang.upper()}-{task_type}...")
    trainer.train()

    # SAVE ONLY THE FINAL MODEL TO LOCAL FIRST
    final_path = f"/content/final_model_{lang}_{task_type}"
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)
    print(f"✅ Finished {lang}-{task_type}")
    return final_path

# --- EXECUTE THE RESCUE PLAN ---
# We train sequentially. If one fails, we at least have the others.
model_paths = {}

# 1. Train English (Task A) - Priority Fix
model_paths['en_A'] = train_fast_model('en', 'A')

# 2. Train Spanish (Task A)
model_paths['es_A'] = train_fast_model('es', 'A')

# 3. Train Italian (Task B)
model_paths['it_B'] = train_fast_model('it', 'B')

# 4. Train Spanish (Task B)
model_paths['es_B'] = train_fast_model('es', 'B')

print("\n🎉 ALL MODELS TRAINED LOCALLY! READY FOR INFERENCE.")

🚀 Downloading distilbert-base-multilingual-cased to LOCAL disk (/content/local_model_cache)...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/399 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/909M [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/911M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import torch
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch import nn

# --- CONFIGURATION ---
os.environ["WANDB_DISABLED"] = "true"
MODEL_ID = "distilbert-base-multilingual-cased"
LOCAL_MODEL_PATH = "/content/local_model_cache"
BATCH_SIZE = 16
EPOCHS = 2          # Reduced to 2 to ensure you finish by 3 PM
LEARNING_RATE = 2e-5
MAX_LEN = 128

# 1. CLEAN & SMART DOWNLOAD (The Fix)
if os.path.exists(LOCAL_MODEL_PATH):
    shutil.rmtree(LOCAL_MODEL_PATH) # Clear corrupt cache

print(f"🚀 Downloading ONLY PyTorch files for {MODEL_ID}...")

# This filter prevents downloading the huge TF and ONNX files
snapshot_download(
    repo_id=MODEL_ID,
    local_dir=LOCAL_MODEL_PATH,
    allow_patterns=["*.json", "*.txt", "*.bin", "*.safetensors"], # Only essentials
    ignore_patterns=["*.h5", "*.onnx", "*.ot"] # Block the heavy junk
)
print("✅ Download Complete! (Lightweight version)")

# 2. Data Preparation
def prepare_data(lang, task_type='A'):
    try:
        df = pd.read_csv(f'train_{lang}.csv')
    except:
        # Emergency path fallback
        df = pd.read_csv(f'./sample_data/train_{lang}.csv')

    df['text'] = df['text'].fillna("").astype(str)
    if 'bio' in df.columns:
        df['bio'] = df['bio'].fillna("").astype(str)
    else:
        df['bio'] = ""

    if task_type == 'B':
        print(f"🔗 Task B ({lang}): Text + Bio")
        df['input_text'] = df['text'] + " [SEP] " + df['bio']
    else:
        print(f"📄 Task A ({lang}): Text Only")
        df['input_text'] = df['text']

    return train_test_split(
        df['input_text'].tolist(), df['label'].tolist(),
        test_size=0.2, stratify=df['label'], random_state=42
    )

# 3. Training Function (Fixed 'eval_strategy')
def train_fast_model(lang, task_type):
    train_texts, val_texts, train_labels, val_labels = prepare_data(lang, task_type)

    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
    train_enc = tokenizer(train_texts, padding=True, truncation=True, max_length=MAX_LEN)
    val_enc = tokenizer(val_texts, padding=True, truncation=True, max_length=MAX_LEN)

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, enc, lab): self.enc, self.lab = enc, lab
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.enc.items()}
            item['labels'] = torch.tensor(self.lab[idx])
            return item
        def __len__(self): return len(self.lab)

    train_ds = MyDataset(train_enc, train_labels)
    val_ds = MyDataset(val_enc, val_labels)

    # Weights
    counts = np.bincount(train_labels)
    if lang == 'en': weights = torch.tensor([1.0, 12.0], dtype=torch.float)
    else: weights = torch.tensor([len(train_labels) / (2 * c) for c in counts], dtype=torch.float)

    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = nn.CrossEntropyLoss(weight=weights.to(model.device))
            return (loss_fct(logits.view(-1, 2), labels.view(-1)), outputs) if return_outputs else loss_fct(logits.view(-1, 2), labels.view(-1))

    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_PATH, num_labels=2)

    output_dir = f'./results_{lang}_{task_type}'
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        save_strategy="no",
        eval_strategy="epoch",  # <--- FIXED KEYWORD HERE
        learning_rate=LEARNING_RATE,
        report_to="none"
    )

    trainer = WeightedTrainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')})

    print(f"🏃 Training {lang.upper()}-{task_type}...")
    trainer.train()

    # Save to local disk for inference script
    final_path = f"/content/final_model_{lang}_{task_type}"
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)
    print(f"✅ Finished {lang}-{task_type}")
    return final_path

# --- EXECUTE SEQUENTIALLY ---
model_paths = {}
# Train English A (Priority)
model_paths['en_A'] = train_fast_model('en', 'A')
# Train Spanish A
model_paths['es_A'] = train_fast_model('es', 'A')
# Train Italian B
model_paths['it_B'] = train_fast_model('it', 'B')
# Train Spanish B
model_paths['es_B'] = train_fast_model('es', 'B')

print("\n🎉 ALL MODELS TRAINED! Run the inference script now.")

🚀 Downloading ONLY PyTorch files for distilbert-base-multilingual-cased...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import torch
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch import nn

# --- CONFIGURATION ---
os.environ["WANDB_DISABLED"] = "true"
MODEL_ID = "distilbert-base-multilingual-cased"
LOCAL_MODEL_PATH = "/content/local_model_cache"
BATCH_SIZE = 16
EPOCHS = 2
LEARNING_RATE = 2e-5
MAX_LEN = 128

# 1. DOWNLOAD FIX: ONLY ALLOW THE FILE THAT IS ALREADY DONE
print(f"🚀 finalizing download for {MODEL_ID} (Skipping redundant files)...")
snapshot_download(
    repo_id=MODEL_ID,
    local_dir=LOCAL_MODEL_PATH,
    allow_patterns=["*.json", "*.txt", "*.bin"],  # <--- ONLY .bin (PyTorch)
    ignore_patterns=["*.h5", "*.onnx", "*.ot", "*.safetensors"] # <--- IGNORE the stuck one
)
print("✅ Ready to Train!")

# 2. Data Preparation
def prepare_data(lang, task_type='A'):
    try:
        df = pd.read_csv(f'train_{lang}.csv')
    except:
        df = pd.read_csv(f'./sample_data/train_{lang}.csv')

    df['text'] = df['text'].fillna("").astype(str)
    if 'bio' in df.columns:
        df['bio'] = df['bio'].fillna("").astype(str)
    else:
        df['bio'] = ""

    if task_type == 'B':
        print(f"🔗 Task B ({lang}): Text + Bio")
        df['input_text'] = df['text'] + " [SEP] " + df['bio']
    else:
        print(f"📄 Task A ({lang}): Text Only")
        df['input_text'] = df['text']

    return train_test_split(
        df['input_text'].tolist(), df['label'].tolist(),
        test_size=0.2, stratify=df['label'], random_state=42
    )

# 3. Training Function
def train_fast_model(lang, task_type):
    train_texts, val_texts, train_labels, val_labels = prepare_data(lang, task_type)

    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
    train_enc = tokenizer(train_texts, padding=True, truncation=True, max_length=MAX_LEN)
    val_enc = tokenizer(val_texts, padding=True, truncation=True, max_length=MAX_LEN)

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, enc, lab): self.enc, self.lab = enc, lab
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.enc.items()}
            item['labels'] = torch.tensor(self.lab[idx])
            return item
        def __len__(self): return len(self.lab)

    train_ds = MyDataset(train_enc, train_labels)
    val_ds = MyDataset(val_enc, val_labels)

    # Weights
    counts = np.bincount(train_labels)
    if lang == 'en': weights = torch.tensor([1.0, 12.0], dtype=torch.float)
    else: weights = torch.tensor([len(train_labels) / (2 * c) for c in counts], dtype=torch.float)

    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = nn.CrossEntropyLoss(weight=weights.to(model.device))
            return (loss_fct(logits.view(-1, 2), labels.view(-1)), outputs) if return_outputs else loss_fct(logits.view(-1, 2), labels.view(-1))

    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_PATH, num_labels=2)

    output_dir = f'./results_{lang}_{task_type}'
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        save_strategy="no",
        eval_strategy="epoch",
        learning_rate=LEARNING_RATE,
        report_to="none"
    )

    trainer = WeightedTrainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')})

    print(f"🏃 Training {lang.upper()}-{task_type}...")
    trainer.train()

    final_path = f"/content/final_model_{lang}_{task_type}"
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)
    print(f"✅ Finished {lang}-{task_type}")
    return final_path

# --- EXECUTE SEQUENTIALLY ---
model_paths = {}
model_paths['en_A'] = train_fast_model('en', 'A')
model_paths['es_A'] = train_fast_model('es', 'A')
model_paths['it_B'] = train_fast_model('it', 'B')
model_paths['es_B'] = train_fast_model('es', 'B')

print("\n🎉 ALL MODELS TRAINED! Run the inference script now.")

🚀 finalizing download for distilbert-base-multilingual-cased (Skipping redundant files)...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Ready to Train!
📄 Task A (en): Text Only


The tokenizer you are loading from '/content/local_model_cache' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/local_model_cache and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🏃 Training EN-A...


...

✅ Finished it-B
🔗 Task B (es): Text + Bio


The tokenizer you are loading from '/content/local_model_cache' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/local_model_cache and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🏃 Training ES-B...


Epoch,Training Loss,Validation Loss,Macro F1
1,No log,0.663900,0.458948
2,No log,0.617858,0.575758


✅ Finished es-B

🎉 ALL MODELS TRAINED! Run the inference script now.


In [ ]:
import pandas as pd
import csv
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import DataLoader, Dataset
import os
from tqdm import tqdm

# --- CONFIGURATION ---
TEAM_NAME = "YourTeamName"  # <--- 🚨 REPLACE THIS WITH YOUR TEAM NAME!
RUN_NUMBER = "1"
MAX_LEN = 128
TEST_DATA_PATH = "./test_data"

# Define tasks and model paths
tasks = [
    {'lang': 'it', 'task': 'A', 'model_path': '/content/final_model_it_A', 'test_file': f'{TEST_DATA_PATH}/it_test.csv'},
    {'lang': 'en', 'task': 'A', 'model_path': '/content/final_model_en_A', 'test_file': f'{TEST_DATA_PATH}/en_test.csv'},
    {'lang': 'es', 'task': 'A', 'model_path': '/content/final_model_es_A', 'test_file': f'{TEST_DATA_PATH}/es_test.csv'},
    {'lang': 'it', 'task': 'B', 'model_path': '/content/final_model_it_B', 'test_file': f'{TEST_DATA_PATH}/it_test.csv'},
    {'lang': 'es', 'task': 'B', 'model_path': '/content/final_model_es_B', 'test_file': f'{TEST_DATA_PATH}/es_test.csv'}
]

# Simple Dataset Class
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

def generate_run(t):
    print(f"\n🔮 Predicting {t['lang'].upper()} Task {t['task']}...")

    # 1. Load Data
    try:
        try: df = pd.read_csv(t['test_file'])
        except: df = pd.read_csv(t['test_file'], sep='\t')
        print(f"   Loaded {len(df)} rows.")
    except FileNotFoundError:
        print(f"❌ ERROR: File {t['test_file']} not found.")
        return

    # 2. Prepare Text
    df['text'] = df['text'].fillna("").astype(str)
    if t['task'] == 'B':
        if 'bio' not in df.columns: df['bio'] = ""
        df['input_text'] = df['text'] + " [SEP] " + df['bio'].fillna("").astype(str)
    else:
        df['input_text'] = df['text']

    # 3. Load Model & Tokenizer (BARE METAL MODE)
    try:
        # We explicitly load config to avoid the dict error
        config = AutoConfig.from_pretrained(t['model_path'])
        model = AutoModelForSequenceClassification.from_pretrained(t['model_path'], config=config)
        tokenizer = AutoTokenizer.from_pretrained(t['model_path'])

        # Move to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()

    except Exception as e:
        print(f"⚠️ Error loading model: {e}")
        return

    # 4. Run Inference Loop
    dataset = InferenceDataset(df['input_text'].tolist(), tokenizer, MAX_LEN)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    all_predictions = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing"):
            input_ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_predictions.extend(preds)

    # 5. Save Results
    filename = f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_{t['lang']}_{t['task']}.tsv"

    # Ensure ID column exists
    id_col = 'id' if 'id' in df.columns else df.columns[0]

    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        # Using standard format: ID [tab] LABEL
        # (Writing without header row first to be safe, or check specific instructions if header needed)
        # Guidelines usually imply no header for Evalita, but we will follow the "id label" example if unsure.
        # Let's write JUST data rows if that's safer, or header if you prefer.
        # I'll include header based on your previous attempts.
        writer.writerow(['id', 'label'])

        for pid, lbl in zip(df[id_col], all_predictions):
            writer.writerow([pid, lbl])

    print(f"✅ Saved: {filename}")

# Run everything
for t in tasks:
    generate_run(t)

print("\n🚀 DONE! DOWNLOAD YOUR FILES NOW!")


🔮 Predicting IT Task A...
   Loaded 725 rows.
⚠️ Error loading model: 'dict' object has no attribute 'model_type'

🔮 Predicting EN Task A...
   Loaded 685 rows.
⚠️ Error loading model: 'dict' object has no attribute 'model_type'

🔮 Predicting ES Task A...
   Loaded 585 rows.
⚠️ Error loading model: 'dict' object has no attribute 'model_type'

🔮 Predicting IT Task B...
   Loaded 725 rows.
⚠️ Error loading model: 'dict' object has no attribute 'model_type'

🔮 Predicting ES Task B...
   Loaded 585 rows.
⚠️ Error loading model: 'dict' object has no attribute 'model_type'

🚀 DONE! DOWNLOAD YOUR FILES NOW!


In [ ]:
import pandas as pd
import csv
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# --- CONFIGURATION ---
TEAM_NAME = "NetGuardAI"  # <--- 🚨 REPLACE WITH YOUR TEAM NAME
RUN_NUMBER = "1"
MAX_LEN = 128
BATCH_SIZE = 16
TEST_DATA_PATH = "./test_data"
# Emergency fallback path for training data
TRAIN_DATA_PATH = "./sample_data"

# Use DistilBERT specific classes to avoid AutoConfig ambiguity
MODEL_NAME = "distilbert-base-multilingual-cased"

# --- DATASET CLASS ---
class EasyDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

# --- MAIN LOOP ---
tasks = [
    {'lang': 'it', 'task': 'A', 'train_file': 'train_it.csv', 'test_file': 'it_test.csv'},
    {'lang': 'en', 'task': 'A', 'train_file': 'train_en.csv', 'test_file': 'en_test.csv'},
    {'lang': 'es', 'task': 'A', 'train_file': 'train_es.csv', 'test_file': 'es_test.csv'},
    {'lang': 'it', 'task': 'B', 'train_file': 'train_it.csv', 'test_file': 'it_test.csv'},
    {'lang': 'es', 'task': 'B', 'train_file': 'train_es.csv', 'test_file': 'es_test.csv'}
]

print("🚀 STARTING EMERGENCY TRAIN-AND-PREDICT LOOP...")

for t in tasks:
    print(f"\n⚡ Processing {t['lang'].upper()} - Task {t['task']}...")

    # 1. LOAD TRAINING DATA
    try:
        # Try local first, then drive
        try: df_train = pd.read_csv(t['train_file'])
        except: df_train = pd.read_csv(f"{TRAIN_DATA_PATH}/{t['train_file']}")

        df_train['text'] = df_train['text'].fillna("").astype(str)
        if t['task'] == 'B':
            df_train['bio'] = df_train['bio'].fillna("").astype(str)
            df_train['input_text'] = df_train['text'] + " [SEP] " + df_train['bio']
        else:
            df_train['input_text'] = df_train['text']

    except Exception as e:
        print(f"❌ Could not load training data for {t['lang']}: {e}")
        continue

    # 2. LOAD TEST DATA
    try:
        test_path = f"{TEST_DATA_PATH}/{t['test_file']}"
        try: df_test = pd.read_csv(test_path)
        except: df_test = pd.read_csv(test_path, sep='\t')

        df_test['text'] = df_test['text'].fillna("").astype(str)
        if t['task'] == 'B':
            if 'bio' not in df_test.columns: df_test['bio'] = ""
            df_test['input_text'] = df_test['text'] + " [SEP] " + df_test['bio'].fillna("").astype(str)
        else:
            df_test['input_text'] = df_test['text']

    except Exception as e:
        print(f"❌ Could not load test data {test_path}: {e}")
        continue

    # 3. TOKENIZE (On the fly)
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)
    train_enc = tokenizer(df_train['input_text'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)
    test_enc = tokenizer(df_test['input_text'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)

    train_dataset = EasyDataset(train_enc, df_train['label'].tolist())
    test_dataset = EasyDataset(test_enc) # No labels for test

    # 4. TRAIN (1 Epoch for Speed)
    # Weighted loss setup
    labels = df_train['label'].tolist()
    counts = np.bincount(labels)
    # Aggressive weights for English, standard for others
    if t['lang'] == 'en': weights = torch.tensor([1.0, 10.0], dtype=torch.float)
    else: weights = torch.tensor([len(labels) / (2 * c) for c in counts], dtype=torch.float)

    class WeightedTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get("logits")
            loss_fct = torch.nn.CrossEntropyLoss(weight=weights.to(model.device))
            return (loss_fct(logits.view(-1, 2), labels.view(-1)), outputs) if return_outputs else loss_fct(logits.view(-1, 2), labels.view(-1))

    model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

    args = TrainingArguments(
        output_dir=f"./temp_{t['lang']}_{t['task']}",
        num_train_epochs=1,          # <--- 1 Epoch is enough for a baseline run!
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE*2,
        learning_rate=2e-5,
        save_strategy="no",
        report_to="none"
    )

    trainer = WeightedTrainer(model=model, args=args, train_dataset=train_dataset)
    trainer.train()

    # 5. PREDICT (Immediately from memory)
    print("   🔮 Predicting...")
    preds_output = trainer.predict(test_dataset)
    pred_labels = np.argmax(preds_output.predictions, axis=1)

    # 6. SAVE TSV
    filename = f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_{t['lang']}_{t['task']}.tsv"

    # Ensure we grab the ID column correctly
    id_col = 'id' if 'id' in df_test.columns else df_test.columns[0]

    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['id', 'label'])
        for pid, lbl in zip(df_test[id_col], pred_labels):
            writer.writerow([pid, lbl])

    print(f"✅ Created: {filename}")

    # Cleanup to save RAM
    del model
    del trainer
    torch.cuda.empty_cache()

print("\n🏁 ALL DONE! Download your files.")

🚀 STARTING EMERGENCY TRAIN-AND-PREDICT LOOP...

⚡ Processing IT - Task A...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


   🔮 Predicting...


✅ Created: multipride2025_NetGuardAI_Run1_it_A.tsv

⚡ Processing EN - Task A...


...

✅ Created: multipride2025_NetGuardAI_Run1_it_B.tsv

⚡ Processing ES - Task B...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


   🔮 Predicting...


✅ Created: multipride2025_NetGuardAI_Run1_es_B.tsv

🏁 ALL DONE! Download your files.


In [ ]:
import pandas as pd
import os
import csv

# --- CONFIGURATION ---
TEAM_NAME = "NetGuardAI"  # Ensure this matches the name in your existing files
RUN_NUMBER = "1"

# The 5 expected filenames based on our previous step
# (If you renamed them, change these lines to match your files!)
file_list = [
    {'filename': f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_it_A.tsv", 'lang': 'it'},
    {'filename': f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_en_A.tsv", 'lang': 'en'},
    {'filename': f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_es_A.tsv", 'lang': 'es'},
    {'filename': f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_it_B.tsv", 'lang': 'it'},
    {'filename': f"multipride2025_{TEAM_NAME}_Run{RUN_NUMBER}_es_B.tsv", 'lang': 'es'}
]

print("🛠️ STARTING FILE REFORMATTING...")

for item in file_list:
    fname = item['filename']
    lang = item['lang']

    if os.path.exists(fname):
        print(f"   Processing {fname}...")

        # 1. Read the existing TSV
        df = pd.read_csv(fname, sep='\t')

        # 2. Add the 'lang' column
        # The professor asked for: id label lang
        df['lang'] = lang

        # 3. Save it back (overwrite or save as new)
        # We ensure the order is: id, label, lang
        # We assume the ID column is named 'id' (from previous script)
        df = df[['id', 'label', 'lang']]

        # Save without index, with header (as requested) or without?
        # The professor said "They should have 3 columns: id label lang"
        # usually implies a header, but Evalita often asks for no header.
        # However, to be safe and clear based on his message, we will keep the header
        # unless you know for sure otherwise.

        # Overwriting the file with the fix
        df.to_csv(fname, sep='\t', index=False)
        print(f"✅ Fixed & Saved: {fname}")

    else:
        print(f"❌ ERROR: Could not find {fname}. Did you rename them? Upload them to Colab if needed.")

print("\n🚀 ALL FILES FIXED! Download them again and resend.")

🛠️ STARTING FILE REFORMATTING...
   Processing multipride2025_NetGuardAI_Run1_it_A.tsv...
✅ Fixed & Saved: multipride2025_NetGuardAI_Run1_it_A.tsv
   Processing multipride2025_NetGuardAI_Run1_en_A.tsv...
✅ Fixed & Saved: multipride2025_NetGuardAI_Run1_en_A.tsv
   Processing multipride2025_NetGuardAI_Run1_es_A.tsv...
✅ Fixed & Saved: multipride2025_NetGuardAI_Run1_es_A.tsv
   Processing multipride2025_NetGuardAI_Run1_it_B.tsv...
✅ Fixed & Saved: multipride2025_NetGuardAI_Run1_it_B.tsv
   Processing multipride2025_NetGuardAI_Run1_es_B.tsv...
✅ Fixed & Saved: multipride2025_NetGuardAI_Run1_es_B.tsv

🚀 ALL FILES FIXED! Download them again and resend.


# Summary of Approach and Model Analysis

### **1. Approach Summary**
To solve the task of detecting reclaimed slurs in multilingual tweets, the following pipeline was executed:
- **Preprocessing**: Data was cleaned (HTML removal), and for **Task B**, the user's bio was concatenated with the tweet text (`text + [SEP] + bio`) to provide context.
- **Handling Imbalance**: Class weights were calculated (inverse frequency) and applied to the Loss function to force the model to pay attention to the minority "reclaimed" class.
- **Modeling**: We utilized a pre-trained **Transformer** model for sequence classification.
- **Emergency Optimization**: Due to Google Colab resource limits (timeouts and disk space), we switched from `XLM-Roberta` to a lighter, faster model and reduced training time.

### **2. The Model**
- **Architecture**: `distilbert-base-multilingual-cased`
- **Why**: It is a distilled version of BERT, retaining 97% of performance while being 40% smaller and 60% faster. This was crucial for finishing the run within the session limits.

### **3. Hyperparameters**
- **Epochs**: `1` (Reduced from 4 to ensure all languages finished)
- **Batch Size**: `16`
- **Learning Rate**: `2e-5`
- **Max Sequence Length**: `128`
- **Loss Function**: Weighted Cross Entropy

### **4. Critical Analysis**

**Arguments AGAINST this approach (Criticism):**
- **Simplicity**: 1 Epoch is often insufficient for the model to converge optimally; it likely underfits.
- **Model Size**: DistilBERT is less powerful than `XLM-Roberta-Large` or `BERT-Large` and may miss subtle nuances in context.
- **No Validation Loop**: In the final "emergency" run, we skipped proper validation monitoring to save time.
- **Basic Weighting**: We used simple inverse class frequency weights. More advanced techniques like Focal Loss or synthetic data augmentation (which we attempted but had to skip) might yield better recall.

**Arguments FOR this approach (Reasonable):**
- **Resource Efficiency**: It fits within the memory and time constraints of a standard Colab environment without crashing.
- **Baseline Effectiveness**: Pre-trained multilingual BERT models are surprisingly robust even with few epochs (few-shot learning capabilities).
- **Context Awareness**: For Task B, explicitly modeling the Bio is a strong feature engineering decision that likely outperforms a complex model looking at text alone.
- **Imbalance Handling**: Even simple weighting is significantly better than standard training for a 1:10 class imbalance.

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.metrics import f1_score

# --- 1. Define Dice Loss Calculation ---
# Dice Loss = 1 - (2 * Intersection) / (Union)
# In binary classification terms: 1 - (2*TP) / (2*TP + FP + FN)
def calculate_dice_loss(preds, labels):
    # Apply softmax to get probabilities, then argmax for hard predictions
    # Or use probabilities for "Soft Dice Loss". Here we use Hard Dice for the metric.
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    # Calculate intersection (True Positives)
    intersection = np.sum(pred_flat * labels_flat)

    # Calculate denominator
    denominator = np.sum(pred_flat) + np.sum(labels_flat)

    # Dice Coefficient
    dice_coeff = (2. * intersection) / (denominator + 1e-6) # Add epsilon to avoid divide by zero
    return 1 - dice_coeff

# --- 2. Re-Train Italian Task A (Fast) ---
print("🔄 Re-training Italian Task A for 1 Epoch to measure Training Dice Loss...")

# Load Data
try:
    df_train = pd.read_csv('train_it.csv')
except:
    df_train = pd.read_csv('./sample_data/train_it.csv')

df_train['text'] = df_train['text'].fillna("").astype(str)
labels = df_train['label'].tolist()

# Tokenize
model_name = "distilbert-base-multilingual-cased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
encodings = tokenizer(df_train['text'].tolist(), truncation=True, padding=True, max_length=128)

# Dataset
class EasyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

dataset = EasyDataset(encodings, labels)

# Class Weights
counts = np.bincount(labels)
weights = torch.tensor([len(labels) / (2 * c) for c in counts], dtype=torch.float)

# Trainer with Weighted Loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights.to(model.device))
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir='./temp_dice_check',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    report_to="none"
)

trainer = WeightedTrainer(model=model, args=training_args, train_dataset=dataset)
trainer.train()

# --- 3. Measure Dice Loss on Training Set ---
print("\n📊 Calculating Metrics on Training Set...")
preds_output = trainer.predict(dataset)
logits = preds_output.predictions

# Calculate Dice Loss
dice_loss = calculate_dice_loss(logits, np.array(labels))
f1 = f1_score(labels, np.argmax(logits, axis=1), average='macro')

print(f"\n🏆 Results on Training Set:")
print(f"   - Dice Loss: {dice_loss:.4f}")
print(f"   - Dice Coefficient: {1-dice_loss:.4f}")
print(f"   - Macro F1: {f1:.4f}")

🔄 Re-training Italian Task A for 1 Epoch to measure Training Dice Loss...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss



📊 Calculating Metrics on Training Set...



🏆 Results on Training Set:
   - Dice Loss: 0.2543
   - Dice Coefficient: 0.7457
   - Macro F1: 0.8383


# Task
Perform NMF Topic Modeling on the "Reclaimed" (label 1) text from the English, Spanish, and Italian training datasets (located at `"./sample_data/"`) to extract the top 10 descriptive keywords for each language. Additionally, retrieve and print the library versions for `transformers`, `torch`, and `scikit-learn`, as well as the GPU model name, to document the experimental setup.

## Load and Compute Text Statistics

### Subtask:
Load the datasets and compute descriptive statistics (mean, max, average length) grouped by class and language.


**Reasoning**:
Load the datasets, compute word counts for the text column, and calculate descriptive statistics (mean, max, std) grouped by the label column to analyze text length differences between classes.



In [ ]:
import pandas as pd
import numpy as np

# Define file paths
# Please ensure the dataset files are placed in the same directory or update this path.
file_paths = {
    'English': './data/train_en.csv'',
    'Spanish': './data/train_es.csv'',
    'Italian': './data/train_it.csv''
}

# Process each dataset
for lang, path in file_paths.items():
    try:
        print(f"--- {lang} Dataset Statistics ---")
        df = pd.read_csv(path)

        # Create word_count column
        df['word_count'] = df['text'].astype(str).apply(lambda x: len(x.split()))

        # Calculate statistics grouped by label
        stats = df.groupby('label')['word_count'].agg(['mean', 'max', 'std'])
        print(stats)
        print("\n")

    except FileNotFoundError:
        print(f"File not found: {path}")

--- English Dataset Statistics ---
            mean  max        std
label                           
0      38.243070  148  25.095986
1      36.681818   92  20.037634


--- Spanish Dataset Statistics ---
            mean  max        std
label                           
0      34.672948   61  11.458676
1      26.714286   54  13.668462


--- Italian Dataset Statistics ---
            mean  max        std
label                           
0      36.782708   57   9.200469
1      22.541063   55  11.808628




## Perform Topic Modeling

### Subtask:
Extract the top 10 descriptive keywords from the 'Reclaimed' (label 1) text in English, Spanish, and Italian using NMF.


**Reasoning**:
Import TfidfVectorizer and NMF, then iterate through each language dataset to filter for 'Reclaimed' text, apply TF-IDF (using stop words for English and frequency filtering for others), fit an NMF model, and extract the top 10 descriptive keywords.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Define file paths (re-stating for robustness)
file_paths = {
    'English': './sample_data/train_en.csv',
    'Spanish': './sample_data/train_es.csv',
    'Italian': './sample_data/train_it.csv'
}

for lang, path in file_paths.items():
    try:
        # a. Load the CSV file
        df = pd.read_csv(path)

        # b. Filter the DataFrame to include only rows where label is 1 (Reclaimed)
        df_reclaimed = df[df['label'] == 1].copy()

        # Handle case with no data
        if df_reclaimed.empty:
            print(f"No 'Reclaimed' data found for {lang}.")
            continue

        # c. Initialize TfidfVectorizer
        if lang == 'English':
            # Use stop_words='english' for English
            vectorizer = TfidfVectorizer(stop_words='english')
        else:
            # For Spanish and Italian, rely on min_df and max_df
            vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)

        # d. Fit and transform
        # Fill NaNs with empty string just in case
        X = vectorizer.fit_transform(df_reclaimed['text'].fillna(''))

        # e. Initialize NMF with n_components=1
        nmf = NMF(n_components=1, random_state=42)

        # f. Fit the NMF model
        nmf.fit(X)

        # g. Retrieve feature names
        feature_names = vectorizer.get_feature_names_out()

        # h. Extract weights from the single component
        component_weights = nmf.components_[0]

        # i. Identify top 10 words
        top_indices = component_weights.argsort()[::-1][:10]
        top_words = feature_names[top_indices]

        print(f"Top 10 descriptive keywords for {lang} (Reclaimed):")
        print(list(top_words))
        print("-" * 40)

    except Exception as e:
        print(f"Error processing {lang} dataset: {e}")

Top 10 descriptive keywords for English (Reclaimed):
['gay', 'faggot', 'people', 'like', 'word', 'queer', 'use', 'time', 'know', 'friends']
----------------------------------------
Top 10 descriptive keywords for Spanish (Reclaimed):
['de', 'que', 'la', 'user', 'orgullolgtbi', 'url', 'en', 'orgullo', 'los', 'no']
----------------------------------------
Top 10 descriptive keywords for Italian (Reclaimed):
['di', 'che', 'sono', 'non', 'la', 'user', 'url', 'un', 'ma', 'per']
----------------------------------------


**Reasoning**:
Retrieve and print the versions of the transformers, torch, and scikit-learn libraries, as well as the GPU model name, to document the experimental setup as requested.



In [ ]:
import transformers
import torch
import sklearn

print(f"Transformers version: {transformers.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

if torch.cuda.is_available():
    print(f"GPU Model: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available")

Transformers version: 4.57.3
Torch version: 2.9.0+cu126
Scikit-learn version: 1.6.1
GPU Model: Tesla T4


## Final Task

### Subtask:
Compile and print a final report summarizing statistics, topic modeling, methodology, and setup.


## Summary:

### Q&A

**Question:** What are the top descriptive keywords for the "Reclaimed" (label 1) text in English, Spanish, and Italian, and what is the experimental setup?

**Answer:**
*   **English Keywords:** The analysis successfully identified terms such as "gay", "faggot", "queer", and "people".
*   **Spanish Keywords:** The results contained relevant terms like "orgullolgtbi" and "orgullo", but were heavily dominated by stopwords (e.g., "de", "que", "la").
*   **Italian Keywords:** The results were primarily composed of stopwords (e.g., "di", "che", "sono"), indicating insufficient filtering.
*   **Experimental Setup:**
    *   **Transformers:** 4.57.3
    *   **Torch:** 2.9.0+cu126
    *   **Scikit-learn:** 1.6.1
    *   **GPU:** Tesla T4

### Data Analysis Key Findings

*   **Text Length Statistics:**
    *   **English:** The average text length is balanced between Label 0 (~38.2 words) and Label 1 (~36.7 words).
    *   **Spanish:** Label 0 texts (~34.7 words) are notably longer than Label 1 texts (~26.7 words).
    *   **Italian:** There is a significant disparity, with Label 0 (~36.8 words) being much longer than Label 1 (~22.5 words).
*   **Topic Modeling Performance:**
    *   The NMF model (n=1) worked effectively for English using the built-in 'english' stopword list.
    *   For Spanish and Italian, relying solely on `min_df=2` and `max_df=0.95` was insufficient for removing high-frequency grammatical articles and prepositions.

### Insights or Next Steps

*   **Improve Stopword Removal:** To extract meaningful topics for Spanish and Italian, explicit language-specific stopword lists (e.g., from NLTK or spaCy) must be integrated into the `TfidfVectorizer`, as frequency-based filtering alone failed to remove common function words.
*   **Investigate Length Disparity:** The significant shortness of "Reclaimed" text in the Italian dataset (avg 22.5 words) compared to others suggests these entries may be short slogans or hashtags; this structural difference should be considered during model training.
